In [1]:
import sys
sys.path.append(r'D:\Fausto Stangler\Documentos\Python\DSH')
import assets.helper as b3
import assets.functions as run




In [2]:
import assets.helper as b4
import assets.functions as run4


In [ ]:
import assets.helper as b3
import assets.functions as run



In [3]:
import os
import datetime
import time
import winsound
import random

import pandas as pd

from google.cloud import storage
import json
import gzip
import io
import base64

import requests
from bs4 import BeautifulSoup

import unidecode
import string
import re

### Convert

In [24]:
df_rad_web = run4.load_parquet('rad_web')


load rad_web


In [ ]:
cols_nsd = ['companhia', 'trimestre', 'versao', 'dri', 'dri2', 'dre', 'auditor', 'auditor_rt', 'cancelamento', 'protocolo', 'envio', 'url', 'nsd']
nsd = nsd[cols_nsd]
nsd = run4.save_parquet(nsd, 'nsd')


In [ ]:
mask = nsd['dre'].isin(b3.dfp)


In [27]:
b3.bin_size/5


10.0

In [21]:
df_rad_web['conta'].dtype


dtype('O')

In [23]:
m = df_rad_web['conta'].isnull()
df_rad_web[m]

,companhia,trimestre,conta,descrição,valor,unidade,demo_det,demo_fin,url


In [ ]:
# Assuming df_nsd is your DataFrame
df_acoes.columns = df_acoes.columns.str.lower()


In [ ]:
df_acoes = run4.save_parquet(df_acoes, 'acoes')

In [ ]:
df_nsd.rename(columns={
    'company': 'Companhia', 
    'data': 'Trimestre', 
    'url': 'URL', 
    }, inplace=True)


In [ ]:
df_nsd = run4.save_parquet(df_nsd, 'nsd', upload=True)


### Clean Acoes

In [ ]:
df_nsd = run.load_parquet('nsd').reset_index(drop=True)
df_nsd['Trimestre'] = pd.to_datetime(df_nsd['Trimestre'], errors='coerce', format='%d%m%Y')

# set_index
df_nsd['nsd'] = df_nsd['URL'].apply(lambda x: int(x.split('Documento=')[-1].split('&')[0]))
# df_nsd = df_nsd.set_index('nsd')

# Filter, Deduplicate and keep last subset
df_nsd_filtered = df_nsd[df_nsd['dre'].isin(b3.dfp)].copy()
df_nsd_filtered = df_nsd_filtered.sort_values(by=['Companhia', 'Trimestre', 'URL'], ascending=[True, True, True])
df_nsd_filtered = df_nsd_filtered.drop_duplicates(subset=['Companhia', 'Trimestre'], keep='last')
df_nsd_filtered.reset_index(drop=True, inplace=True)

df_acoes = run.load_parquet('acoes').reset_index(drop=True)
df_acoes['Trimestre'] = pd.to_datetime(df_acoes['Trimestre'], errors='coerce', format='%d%m%Y')

# Deduplicate and keep last subset
df_acoes = df_acoes.sort_values(by=['Companhia', 'Trimestre', 'URL'], ascending=[True, True, True])
df_acoes = df_acoes.drop_duplicates(subset=['Companhia', 'Trimestre'], keep='last')
df_acoes.reset_index(drop=True, inplace=True)

# set_index
df_acoes['nsd'] = df_acoes['URL'].apply(lambda x: int(x.split('Documento=')[-1].split('&')[0]))
# df_acoes = df_acoes.set_index('nsd')

# Combine rows based on Company and Quarter
df_acoes = pd.DataFrame(columns=b3.columns['acoes'])

df_acoes_nsd = pd.merge(df_acoes, df_nsd_filtered, on=['Companhia', 'Trimestre'], how='outer', suffixes=('', '_nsd'), indicator=True)



In [ ]:
df_nsd

In [ ]:

linhas_para_atualizar = df_acoes_nsd[df_acoes_nsd['nsd_nsd'] > df_acoes_nsd['nsd']]


In [ ]:

linhas_para_atualizar[['Companhia', 'Trimestre', 'URL', 'URL_nsd', 'nsd', 'nsd_nsd']]

In [ ]:
# Obtenha um conjunto de ['Companhia', 'Trimestre'] das linhas a serem atualizadas
conjunto_para_atualizar = set(zip(linhas_para_atualizar['Companhia'], linhas_para_atualizar['Trimestre']))

# Filtre o DataFrame de ações para excluir as linhas correspondentes
df_acoes = df_acoes[~df_acoes.set_index(['Companhia', 'Trimestre']).index.isin(conjunto_para_atualizar)]


In [ ]:
# Crie um DataFrame apenas com as linhas para atualização e as colunas relevantes
linhas_para_atualizar = linhas_para_atualizar[['Companhia', 'Trimestre', 'URL_nsd']]

# Faça a fusão do DataFrame de ações original com o DataFrame de atualização usando 'URL_nsd' como chave
df_acoes = pd.merge(df_acoes, linhas_para_atualizar, on=['Companhia', 'Trimestre'], how='left')

# Renomeie a coluna 'URL_nsd' para 'URL' no DataFrame de ações (se necessário)
df_acoes.rename(columns={'URL_nsd': 'URL'}, inplace=True)
df_acoes

In [ ]:
c = 'ALFA HOLDINGS SA'
t = '2010-12-31'

m_acoes = (df_acoes['Companhia'] == c) & (df_acoes['Trimestre'] == t)
df_acoes[m_acoes]

In [ ]:
m_nsd = (df_nsd_filtered['Companhia'] == c) & (df_nsd_filtered['Trimestre'] == t)
df_nsd_filtered[m_nsd]


In [ ]:
urls_most_recent = acoes_most_recent['URL'].unique()
urls_most_recent

df_nsd_filtered = df_nsd_filtered[~df_nsd_filtered['URL'].isin(urls_most_recent)]


In [ ]:
acoes_most_recent[['Companhia', 'Trimestre', 'URL', 'URL_nsd', 'nsd', 'nsd_nsd']]

In [ ]:
df_nsd_filtered[['Companhia', 'Trimestre', 'nsd']]

In [ ]:
    # Filter for relevant document types
    df_nsd_filtered = df_nsd[df_nsd['dre'].isin(b3.dfp)].copy()

    # Convert data types and remove duplicates
    df_nsd_filtered['nsd'] = df_nsd_filtered['nsd'].astype(int)
    df_nsd_filtered['Trimestre'] = pd.to_datetime(df_nsd_filtered['Trimestre'], format='%d%m%Y', errors='coerce')
    df_nsd_filtered.sort_values('nsd', ascending=True, inplace=True)
    df_nsd_filtered.drop_duplicates(['Companhia', 'Trimestre'], keep='last', inplace=True)

    # Identify latest document per company and quarter
    nsd_max = df_nsd_filtered.groupby(['Companhia', 'Trimestre'])['nsd'].max().reset_index()
    nsd_max['Trimestre'] = pd.to_datetime(nsd_max['Trimestre'], format='%d%m%Y', errors='coerce')
    nsd_max['nsd'] = nsd_max['nsd'].astype(int)

    df_nsd_filtered.sort_values(by=['Companhia', 'Trimestre'], ascending=[True, True], inplace=True)
	
    if not df_acoes.empty:
        # Extract NSD from URLs and convert 'Trimestre' to datetime
        df_acoes['nsd'] = df_acoes['URL'].apply(lambda x: int(x.split('Documento=')[-1].split('&')[0]))
        df_acoes['Trimestre'] = pd.to_datetime(df_acoes['Trimestre'], format='%d%m%Y', errors='coerce')

        # Combine rows based on Company and Quarter, retaining recent entries
        df_acoes_merged = pd.merge(df_acoes, nsd_max, on=['Companhia', 'Trimestre'], how='left', suffixes=('', '_max'))
        acoes_most_recent = df_acoes_merged[df_acoes_merged['nsd'] >= df_acoes_merged['nsd_max']].drop(columns=['nsd_max'])

        # List URLs from recent entries to remove from df_nsd_filtered
        urls_to_remove = acoes_most_recent['URL'].unique()
        df_nsd_filtered = df_nsd_filtered[~df_nsd_filtered['URL'].isin(urls_to_remove)]

        # Drop temporary columns
        df_acoes.drop(columns=['nsd', 'nsd_max'], inplace=True, errors='ignore')
